In [2]:
import os

In [3]:
%pwd

'/workspaces/project/research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'/workspaces/project'

In [26]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    validation_data:Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [27]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [28]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    


    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Chest-CT-Scan-data/Data/train")
        validation_data = os.path.join(self.config.data_ingestion.unzip_dir, "Chest-CT-Scan-data/Data/valid")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            validation_data=Path(validation_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config


In [29]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time
import shutil
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
from tensorflow.keras.applications import resnet

In [30]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def train_valid_generator(self):

        train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(dtype='float32',
                                                                    preprocessing_function=resnet.preprocess_input)
        self.train_generator = train_datagen.flow_from_directory(self.config.training_data,
                                                   batch_size = self.config.params_batch_size,
                                                   target_size = self.config.params_image_size[:-1],
                                                   class_mode = 'categorical')

        valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(dtype='float32',
                                                                    preprocessing_function=resnet.preprocess_input)
        self.valid_generator = valid_datagen.flow_from_directory(self.config.validation_data,
                                                   batch_size = self.config.params_batch_size,
                                                   target_size = self.config.params_image_size[:-1],
                                                   class_mode = 'categorical')

        # if self.config.params_is_augmentation:
        #     train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
        #         rotation_range=40,
        #         horizontal_flip=True,
        #         width_shift_range=0.2,
        #         height_shift_range=0.2,
        #         shear_range=0.2,
        #         zoom_range=0.2,
        #         **datagenerator_kwargs
        #     )
        # else:
        #     train_datagenerator = valid_datagenerator

        # self.train_generator = train_datagenerator.flow_from_directory(
        #     directory=self.config.training_data,
        #     subset="training",
        #     shuffle=True,
        #     **dataflow_kwargs
        # )

    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)
        
        destination_dir = path.parent.parent.parent / "model"
        if not destination_dir.exists():
            destination_dir.mkdir(parents=True)

        # Copy the file to the destination directory
        destination_file = destination_dir / "model.h5"
        shutil.copyfile(path, destination_file)


    
    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.checkpointer = ModelCheckpoint(filepath=str(self.config.trained_model_path),
                            monitor='val_loss', verbose = 1,
                            save_best_only=True)
        self.early_stopping = EarlyStopping(verbose=1, patience=15)
        
        self.model.fit(self.train_generator,
                    steps_per_epoch = self.steps_per_epoch,
                    epochs = self.config.params_epochs,
                    verbose = 1,
                    validation_data = self.valid_generator,
                    callbacks = [self.checkpointer, self.early_stopping])

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [31]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

[2024-04-02 23:17:01,107: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-04-02 23:17:01,110: INFO: common: yaml file: params.yaml loaded successfully]
[2024-04-02 23:17:01,111: INFO: common: created directory at: artifacts]
[2024-04-02 23:17:01,111: INFO: common: created directory at: artifacts/training]
Found 613 images belonging to 4 classes.
Found 72 images belonging to 4 classes.
Epoch 1/100
38/38 [==============================] - ETA: 0s - loss: 2.6238 - acc: 0.2814
Epoch 1: val_loss improved from inf to 1.69032, saving model to artifacts/training/model.h5
38/38 [==============================] - 227s 6s/step - loss: 2.6238 - acc: 0.2814 - val_loss: 1.6903 - val_acc: 0.2917
Epoch 2/100
38/38 [==============================] - ETA: 0s - loss: 2.0634 - acc: 0.3769
Epoch 2: val_loss improved from 1.69032 to 1.31077, saving model to artifacts/training/model.h5
38/38 [==============================] - 222s 6s/step - loss: 2.0634 - acc: 0.3769 - val_loss: 1.3108